In [1]:
# import dependencies
import spotipy
import spotipy.util as util
import pandas as pd
import pprint

## Grab csv files dataframe

In [2]:
top_2017 = pd.read_csv("assets/data/top2017.csv", encoding='latin-1')
top_2018 = pd.read_csv("assets/data/top2018.csv", encoding='unicode_escape')
top_2019 = pd.read_csv("assets/data/top2019.csv", encoding='unicode_escape')

### create search list of each year

In [3]:
# initialize list to scrape
search_data = []

# pull tracks and artists
tracks = list(top_2019['Track.Name'])
artists = list(top_2019['Artist.Name'])
# add each list to search_data
for i in range(len(tracks)):
    search_data.append([tracks[i], artists[i]])



In [4]:
def search_by_track(track, artist):

    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by track name 
    result = sp.search(q=track, type='track')
    # save first 10 results
    first_ten_albums = result['tracks']['items']
    # loop through each album 
    for album in first_ten_albums:
        # extract artist of each album
        possible_artist = album['album']['artists'][0]['name']
        # see if album artist matches query artist
        if possible_artist == artist:
            # match found, add to song-ids
            value = [artist, track, result['tracks']['items'][0]['id']]
            return value
    return None

def search_by_artist(artist, track):
    ''' returns correct track by artist to search'''
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
  
    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    top_tracks = sp.artist_top_tracks(artist_id)
    top_10 = top_tracks['tracks']
    # loop through top tracks, check to see if any match track query
    for song in top_10:
        possible_song = song['name']
        # matches enough, match found
        if track[:2] == possible_song[:2]:
            track = possible_song
            return track
    return None

def grab_genre(artist):
    ''' returns a genre of artist'''
    # create spotipy object
    sp = spotipy.Spotify(auth=token)

    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    genre = sp.artist(artist_id)['genres']
    return genre
 

In [5]:
#https://open.spotify.com/user/1298997744?si=ijwp9NZOQoyjO9qOZliclw
SPOTIPY_CLIENT_ID='7655188a18144343b07e44965cc38310'
SPOTIPY_CLIENT_SECRET='2bec9b47853e4ea3a419418a211ec45d'
SPOTIPY_REDIRECT_URI='https://google.com/'
scope = 'user-library-read'
# Grab username
username = "1298997744"

token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

results = []

# Create our spotify object
for query in search_data:
    if token:
        # create track & artist search variables
        track = query[0]
        artist = query[1]
        genre = grab_genre(artist)

        # run search by track to find results
        value = search_by_track(track, artist)

        # if search_by_track returns none, use search_by_artist.
        # else append to results
        if value == None:
            correct_track = search_by_artist(artist, track)
            # run search_by_track again
            value = search_by_track(correct_track, artist)
            value.append(genre)
            results.append(value)
        else:
            value.append(genre)
            results.append(value)   
    else:
        print("Can't get token for", username)

In [7]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

# create spotipy object
sp = spotipy.Spotify(auth=token)

top_2019_data = []
for each in results:
    features = sp.audio_features(each[2])
    data = features[0]
    data['name'] = each[1]
    data['artists'] = each[0]
    data['year'] = '2019'
    data['genre'] = each[3]
    top_2019_data.append(data)

[{'danceability': 0.759,
  'energy': 0.54,
  'key': 9,
  'loudness': -6.039,
  'mode': 0,
  'speechiness': 0.0287,
  'acousticness': 0.037,
  'instrumentalness': 0,
  'liveness': 0.0945,
  'valence': 0.75,
  'tempo': 116.947,
  'type': 'audio_features',
  'id': '0TK2YIli7K1leLovkQiNik',
  'uri': 'spotify:track:0TK2YIli7K1leLovkQiNik',
  'track_href': 'https://api.spotify.com/v1/tracks/0TK2YIli7K1leLovkQiNik',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0TK2YIli7K1leLovkQiNik',
  'duration_ms': 190960,
  'time_signature': 4,
  'name': 'Señorita',
  'artists': 'Shawn Mendes',
  'year': '2019',
  'genre': ['canadian pop', 'pop', 'post-teen pop', 'viral pop']},
 {'danceability': 0.786,
  'energy': 0.808,
  'key': 7,
  'loudness': -3.702,
  'mode': 1,
  'speechiness': 0.0882,
  'acousticness': 0.0846,
  'instrumentalness': 0.000289,
  'liveness': 0.0822,
  'valence': 0.609,
  'tempo': 105.027,
  'type': 'audio_features',
  'id': '2ksOAxtIxY8yElEWw8RhgK',
  'uri': 'spotify:t

In [8]:
top2019_df = pd.DataFrame.from_dict(top_2019_data)
top_2019_df = top2019_df.drop(columns=['type', 'uri','track_href','analysis_url'])
top_2019_df = top_2019_df[['id','name','artists','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','year','genre']]
top_2019_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,0TK2YIli7K1leLovkQiNik,Señorita,Shawn Mendes,0.759,0.540,9,-6.039,0,0.0287,0.03700,0.000000,0.0945,0.750,116.947,190960,4,2019,"[canadian pop, pop, post-teen pop, viral pop]"
1,2ksOAxtIxY8yElEWw8RhgK,China,Anuel AA,0.786,0.808,7,-3.702,1,0.0882,0.08460,0.000289,0.0822,0.609,105.027,301714,4,2019,"[latin, reggaeton flow, trap latino]"
2,0Ryd8975WihbObpp5cPW1t,boyfriend (with Social House),Ariana Grande,0.400,0.795,10,-3.731,0,0.4610,0.11900,0.000000,0.1590,0.702,190.097,186107,4,2019,"[dance pop, pop, post-teen pop]"
3,4evmHXcjt3bTUHD1cvny97,Beautiful People (feat. Khalid),Ed Sheeran,0.640,0.648,5,-8.113,0,0.1870,0.12400,0.000000,0.0802,0.548,92.977,197867,4,2019,"[pop, uk pop]"
4,0t3ZvGKlmYmVsDzBJAXK8C,Goodbyes (Feat. Young Thug),Post Malone,0.548,0.653,5,-3.875,1,0.0810,0.45600,0.000000,0.1030,0.163,150.068,174853,4,2019,"[dfw rap, melodic rap, rap]"
5,3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),Ed Sheeran,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,219947,4,2019,"[pop, uk pop]"
6,1lOe9qE0vR9zwWQAOk6CoO,Ransom,Lil Tecca,0.745,0.642,7,-6.257,0,0.2870,0.02040,0.000000,0.0658,0.226,179.974,131240,4,2019,[melodic rap]
7,6b2RcmUt1g9N9mQ3CbjX2Y,How Do You Sleep?,Sam Smith,0.477,0.682,1,-4.931,0,0.0925,0.15300,0.000000,0.0763,0.345,110.567,202205,5,2019,"[pop, uk pop]"
8,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,Lil Nas X,0.878,0.619,6,-5.560,1,0.1020,0.05330,0.000000,0.1130,0.639,136.041,157067,4,2019,"[country rap, lgbtq+ hip hop, pop rap]"
9,2Fxmhks0bxGSBdJ92vM42m,bad guy,Billie Eilish,0.701,0.425,7,-10.965,1,0.3750,0.32800,0.130000,0.1000,0.562,135.128,194088,4,2019,"[electropop, pop]"


In [ ]:
#top_2019_df.to_csv('assets/data/top2019fixed.csv', index=False)
top_2019_df.to_excel('assets/data/top2019fixed2.xlsx', index=False)

In [ ]:
final_df = pd.read_excel("assets/data/top2019fixed2.xlsx")
final_df

### finds genre of artist

In [9]:
def add_genre(dataframe):
    '''takes in a dataframe, adds a genre column by artist'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
    # grab artist column for Spotipy search
    artists = list(dataframe['artists'])  
   
    # initialize genre column
    genres = []

    for person in artists:
        # create spotipy object
        sp = spotipy.Spotify(auth=token)
        # search by artist name
        result = sp.search(q=person, type='artist')
        try:
            # grab artist id
            artist_id = result['artists']['items'][0]['id']
            # search artist top tracks
            genre = sp.artist(artist_id)['genres']
            genres.append(genre)
        except IndexError:
            i = dataframe.index[dataframe['artists'] == person]
            song_df = dataframe.loc[i, 'name']
            song_list = song_df.values
            song = song_list[0]
            result = sp.search(q=song, type='track')
            artist = result['tracks']['items'][0]['artists'][0]['name']
            result = sp.search(q=artist, type='artist')
            genre = result['artists']['items'][0]['genres']
            genres.append(genre)
                   
    dataframe['genre'] = genres


In [10]:
add_genre(top_2017)
add_genre(top_2018)

Ti?sto


In [12]:
top_2017[20:70]

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
20,2rb5MvYT7ZIxbKW5hfcHx,Mi Gente,J Balvin,0.543,0.677,11,-4.915,0,0.0993,0.01480,0.000006,0.1300,0.294,103.809,189440,4,2017,"[latin, reggaeton]"
21,0tKcYR2II1VCQWT79i5Nr,Thunder,Imagine Dragons,0.600,0.810,0,-4.749,1,0.0479,0.00683,0.210000,0.1550,0.298,167.880,187147,4,2017,[modern rock]
22,5uCax9HTNlzGybIStD3vD,Say You Won't Let Go,James Arthur,0.358,0.557,10,-7.398,1,0.0590,0.69500,0.000000,0.0902,0.494,85.043,211467,4,2017,"[pop, post-teen pop, talent show, uk pop]"
23,79cuOz3SPQTuFrp8WgftA,There's Nothing Holdin' Me Back,Shawn Mendes,0.857,0.800,2,-4.035,1,0.0583,0.38100,0.000000,0.0913,0.966,121.996,199440,4,2017,"[canadian pop, pop, post-teen pop, viral pop]"
24,6De0lHrwBfPfrhorm9q1X,Me RehÃºso,Danny Ocean,0.744,0.804,1,-6.327,1,0.0677,0.02310,0.000000,0.0494,0.426,104.823,205715,4,2017,"[latin, latin pop, reggaeton, tropical]"
25,6D0b04NJIKfEMg040WioJ,Issues,Julia Michaels,0.706,0.427,8,-6.864,1,0.0879,0.41300,0.000000,0.0609,0.420,113.804,176320,4,2017,"[dance pop, electropop, pop, post-teen pop]"
26,0afhq8XCExXpqazXczTSv,Galway Girl,Ed Sheeran,0.624,0.876,9,-3.374,1,0.1000,0.07350,0.000000,0.3270,0.781,99.943,170827,4,2017,"[pop, uk pop]"
27,3ebXMykcMXOcLeJ9xZ17X,Scared to Be Lonely,Martin Garrix,0.584,0.540,1,-7.786,0,0.0576,0.08950,0.000000,0.2610,0.195,137.972,220883,4,2017,"[big room, edm, pop, progressive house, tropic..."
28,7BKLCZ1jbUBVqRi2FVlTV,Closer,The Chainsmokers,0.748,0.524,8,-5.599,1,0.0338,0.41400,0.000000,0.1110,0.661,95.010,244960,4,2017,"[electropop, pop, tropical house]"
29,1x5sYLZiu9r5E43kMlt9f,Symphony (feat. Zara Larsson),Clean Bandit,0.707,0.629,0,-4.581,0,0.0563,0.25900,0.000016,0.1380,0.457,122.863,212459,4,2017,"[dance pop, edm, pop, post-teen pop, tropical ..."
